## Work
### 請結合前面的知識與程式碼，比較不同的 regularization 的組合對訓練的結果與影響：如 dropout, regularizers, batch-normalization 等

In [1]:
import os
import keras
import itertools
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.regularizers import l1

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
"""Code Here
    建立你的神經網路
"""
def build_mlp(input_shape, output_units = 10, num_neurons = [512, 256, 128], l1_ratio=1e-4, drop_ratio = 0.3):
    
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = BatchNormalization()(input_layer)
            x = keras.layers.Dense(units = n_units,
                                   activation = 'relu',
                                   name = 'hidden_layer'+str(i+1),
                                   kernel_regularizer = l1(l1_ratio))(x)
            x = Dropout(drop_ratio)(x)
        else:
            x = BatchNormalization()(x)
            x = keras.layers.Dense(units = n_units, 
                                   activation = 'relu',
                                   name = 'hidden_layer'+str(i+1), 
                                   kernel_regularizer = l1(l1_ratio))(x)
            x = Dropout(drop_ratio)(x)
    output_layer = keras.layers.Dense(units = output_units, activation = "softmax", name = "output")(x)
    model = keras.models.Model(inputs = [input_layer], outputs = [output_layer])
    return model

In [6]:
"""Code Here
設定超參數
"""
LEARNING_RATE = 1e-3
EPOCHS = 50
BATCH_SIZE = 1024
MOMENTUM = 0.95
L1_EXP = [1e-2, 1e-4, 1e-8, 1e-12]

In [7]:
results = {}
"""Code Here
撰寫你的訓練流程並將結果用 dictionary 紀錄
"""
for regulizer_ratio in L1_EXP:
    keras.backend.clear_session()
    print('Experiment with Regulizer:{:.13f}'.format(regulizer_ratio))
    model = build_mlp(input_shape = x_train.shape[1:], l1_ratio = regulizer_ratio)
    model.summary()
    optimizer = keras.optimizers.SGD(lr = LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
    model.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'], optimizer = optimizer)
    
    model.fit(x_train, y_train, batch_size = BATCH_SIZE, 
              epochs = EPOCHS, validation_data = (x_test, y_test), 
              shuffle = True)
    
    train_loss = model.history.history["loss"]
    valid_loss = model.history.history["val_loss"]
    train_acc = model.history.history["acc"]
    valid_acc = model.history.history["val_acc"]
    
    exp_name_tag = "exp-l1-%s" % str(regulizer_ratio)
    results[exp_name_tag] = {'train-loss': train_loss,
                             'valid-loss': valid_loss,
                             'train-acc': train_acc,
                             'valid-acc': valid_acc}




Experiment with Regulizer:0.0100000000000


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 3072)              12288     
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)      

50000/50000 [==============================] - 9s 189us/step - loss: 2.4741 - acc: 0.2529 - val_loss: 2.4575 - val_acc: 0.2874
Epoch 18/50
50000/50000 [==============================] - 9s 188us/step - loss: 2.4619 - acc: 0.2534 - val_loss: 2.4585 - val_acc: 0.2811
Epoch 19/50
50000/50000 [==============================] - 10s 191us/step - loss: 2.4533 - acc: 0.2533 - val_loss: 2.4329 - val_acc: 0.2786
Epoch 20/50
50000/50000 [==============================] - 10s 192us/step - loss: 2.4560 - acc: 0.2539 - val_loss: 2.4288 - val_acc: 0.2839
Epoch 21/50
50000/50000 [==============================] - 10s 190us/step - loss: 2.4453 - acc: 0.2527 - val_loss: 2.4103 - val_acc: 0.2776
Epoch 22/50
50000/50000 [==============================] - 10s 191us/step - loss: 2.4379 - acc: 0.2540 - val_loss: 2.4317 - val_acc: 0.2733
Epoch 23/50
50000/50000 [==============================] - 9s 190us/step - loss: 2.4340 - acc: 0.2518 - val_loss: 2.4100 - val_acc: 0.2871
Epoch 24/50
50000/50000 [==========

Epoch 12/50
50000/50000 [==============================] - 10s 195us/step - loss: 5.5499 - acc: 0.3780 - val_loss: 5.3733 - val_acc: 0.4432
Epoch 13/50
50000/50000 [==============================] - 10s 191us/step - loss: 5.5188 - acc: 0.3844 - val_loss: 5.3481 - val_acc: 0.4442
Epoch 14/50
50000/50000 [==============================] - 10s 192us/step - loss: 5.4829 - acc: 0.3908 - val_loss: 5.3198 - val_acc: 0.4483
Epoch 15/50
50000/50000 [==============================] - 10s 197us/step - loss: 5.4489 - acc: 0.3980 - val_loss: 5.2933 - val_acc: 0.4534
Epoch 16/50
50000/50000 [==============================] - 10s 196us/step - loss: 5.4224 - acc: 0.3986 - val_loss: 5.2683 - val_acc: 0.4527
Epoch 17/50
50000/50000 [==============================] - 10s 202us/step - loss: 5.3906 - acc: 0.4030 - val_loss: 5.2434 - val_acc: 0.4566
Epoch 18/50
50000/50000 [==============================] - 10s 198us/step - loss: 5.3627 - acc: 0.4080 - val_loss: 5.2189 - val_acc: 0.4569
Epoch 19/50
50000/50

50000/50000 [==============================] - 10s 194us/step - loss: 1.8793 - acc: 0.3367 - val_loss: 1.6321 - val_acc: 0.4192
Epoch 8/50
50000/50000 [==============================] - 10s 195us/step - loss: 1.8472 - acc: 0.3474 - val_loss: 1.6108 - val_acc: 0.4272
Epoch 9/50
50000/50000 [==============================] - 10s 198us/step - loss: 1.8196 - acc: 0.3550 - val_loss: 1.5968 - val_acc: 0.4352
Epoch 10/50
50000/50000 [==============================] - 10s 197us/step - loss: 1.7893 - acc: 0.3653 - val_loss: 1.5815 - val_acc: 0.4408
Epoch 11/50
50000/50000 [==============================] - 10s 198us/step - loss: 1.7707 - acc: 0.3710 - val_loss: 1.5674 - val_acc: 0.4443
Epoch 12/50
50000/50000 [==============================] - 10s 198us/step - loss: 1.7515 - acc: 0.3752 - val_loss: 1.5564 - val_acc: 0.4461
Epoch 13/50
50000/50000 [==============================] - 10s 203us/step - loss: 1.7317 - acc: 0.3812 - val_loss: 1.5448 - val_acc: 0.4504
Epoch 14/50
50000/50000 [=========

50000/50000 [==============================] - 10s 202us/step - loss: 2.2309 - acc: 0.2379 - val_loss: 1.8490 - val_acc: 0.3464
Epoch 3/50
50000/50000 [==============================] - 10s 199us/step - loss: 2.1000 - acc: 0.2720 - val_loss: 1.7753 - val_acc: 0.3746
Epoch 4/50
50000/50000 [==============================] - 10s 200us/step - loss: 2.0176 - acc: 0.2933 - val_loss: 1.7308 - val_acc: 0.3943
Epoch 5/50
50000/50000 [==============================] - 10s 206us/step - loss: 1.9625 - acc: 0.3113 - val_loss: 1.6996 - val_acc: 0.4009
Epoch 6/50
50000/50000 [==============================] - 10s 200us/step - loss: 1.9179 - acc: 0.3190 - val_loss: 1.6719 - val_acc: 0.4138
Epoch 7/50
50000/50000 [==============================] - 10s 199us/step - loss: 1.8731 - acc: 0.3355 - val_loss: 1.6491 - val_acc: 0.4207
Epoch 8/50
50000/50000 [==============================] - 10s 204us/step - loss: 1.8516 - acc: 0.3393 - val_loss: 1.6318 - val_acc: 0.4258
Epoch 9/50
50000/50000 [==============

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
"""Code Here
將結果繪出
"""
color_bar = ['r', 'g', 'b', 'k', 'y', 'm']

plt.figure(figsize = (8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
plt.title('Loss')
plt.legend()
plt.show()

plt.figure(figsize = (8, 6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
plt.title('Accuracy')
plt.legend()
plt.show()